In [1]:
import pandas as pd
import numpy as np

In [2]:
# grab all unique providers 
provider_OH = pd.read_json("https://opendata.fcc.gov/resource/s7ac-qb2r.json?"
                           "$select=logrecno,provider_id,hoconum,blockcode,techcode,maxaddown,maxadup"
                           "&consumer=1&stateabbr='OH'&$limit=5000000"
                          )

In [3]:
# add census tract code
provider_OH["tract"]=provider_OH["blockcode"]//10000

In [6]:
# number of unique providers/holding companies in the state of ohio 
print(provider_OH["provider_id"].nunique(),)
print(provider_OH["hoconum"].nunique(),)

121
100


In [4]:
# count num of providers in each tract
df_tract_provider_count = provider_OH.groupby(["tract", "provider_id"]).count()
provider_count = df_tract_provider_count.groupby("tract")["blockcode"].count()

In [5]:
# count num of providers data processing
new = provider_count.to_frame()
new["GEO.id2"]=new.index
new = new.rename(columns={"blockcode":"provider_count"})

In [6]:
# count num of holding companies in each tract
df_tract_holding_count = provider_OH.groupby(["tract", "hoconum"]).count()
holding_count = df_tract_holding_count.groupby("tract")["blockcode"].count()

In [7]:
# count num of providers data processing
new2 = holding_count.to_frame()
new2["GEO.id2"]=new2.index
new2 = new2.rename(columns={"blockcode":"holding_count"})

In [8]:
census_info = pd.read_csv("/Users/xcheng/Documents/Oberlin/2019Fall/Econ417/IO_DataAnalysis/census_track_ACS_16_OH.csv")

In [9]:
###########
# Verify count
###########
print(len(set(new['GEO.id2']) & set(census_info['GEO.id2'])))
print(len(set(new['GEO.id2'])))
print(len(set(census_info['GEO.id2'])))

2952
2952
2952


In [10]:
# add in the provider count & holding count
census_info = census_info.merge(new, on="GEO.id2")
census_info = census_info.merge(new2, on="GEO.id2")

In [11]:
# census_info.to_csv("simple_combined.csv")

In [12]:
# one line for one provider that provides tech 10 in this census tract
# maxaddown and maxadup are average accross all census block where it provides tech 10 
df_individual = provider_OH[provider_OH['techcode']==10].groupby(["tract", "provider_id"]).mean()

In [13]:
df_individual.reset_index(inplace=True) 

In [14]:
df_individual = df_individual[["tract", "provider_id", "maxaddown", "maxadup"]]

In [20]:
final = pd.merge(df_individual, census_info, left_on="tract", right_on="GEO.id2")

In [22]:
final = final[final['population']>50]

In [23]:
final.to_csv("finally_ready.csv")